### Задание к теме: «Гибридные рекомендательные системы»

In [48]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np


In [49]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [50]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)

строим модель для предсказания рейтинга пользователя

In [51]:
# задаем датасет в формате surprise
dataset = pd.DataFrame({
    'uid': data.userId,
    'iid': data.title,
    'rating': data.rating
})

In [52]:
print(f'min: {dataset.rating.min()}')
print(f'max: {dataset.rating.max()}')

min: 0.5
max: 5.0


In [53]:
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(dataset, reader)

In [54]:
trainset, testset = train_test_split(data_surprise, test_size=.15, random_state=42)

In [55]:
# модель SVD
model = SVD(n_factors=20, n_epochs=20)
model.fit(trainset)

In [56]:
# RMSE
test_pred = model.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8667


0.8667367375454882

In [57]:
# оценка на кросс-валидации
res = cross_validate(model, data_surprise, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8727  0.8684  0.8734  0.8663  0.8700  0.8702  0.0027  
Fit time          1.53    1.52    1.52    1.50    1.53    1.52    0.01    
Test time         0.14    0.09    0.09    0.14    0.15    0.12    0.02    


In [58]:
print(f"{res['test_rmse'].mean()}")

0.8701671131301678


In [59]:
# определяем пользователя и фильмы которые он посмотрел
user = 13
user_movies = data[data.userId == user].title.unique()

In [60]:
# формируем списки по предсказанию и наименовнию фильма
pred_rating = []
titles = []

for movie in data.title.unique():
    if movie in user_movies:
        continue
        
    pred_rating.append(algo.predict(uid=user, iid=movie).est)
    titles.append(movie)

In [61]:
# база фильмов которые пользователь не смотрел и прогнозный рейтинг фильму от пользователя
movies_not_watch = pd.DataFrame({'title': titles, 'pred_r': pred_rating})

In [62]:
# берем топ 10 фильмом из потенциального прогноза пользователя
most_interesting = movies_not_watch.sort_values(by='pred_r', ascending=False)[:10]

In [63]:
# формируем базу фильмов с средним рейтингом по фактическим оценкам пользователя
movies_with_mean_rating = pd.DataFrame(data.groupby('title').rating.mean()).reset_index()

In [64]:
# задаем функцию которая добавит средний рейтинг
def add_mean(row):
    return movies_with_mean_rating[movies_with_mean_rating.title == row].rating.sum()

In [65]:
# в понтенциальные топ 10 добавляем фактический средний рейтинг
most_interesting['mean_rating'] = most_interesting.title.apply(add_mean)

In [66]:
# создаем метрику которая взвешивает два подхода на 70/30 процентов
most_interesting['common_rating'] = most_interesting.pred_r*0.7 + most_interesting.mean_rating*0.3

In [67]:
# формируем топ 5 рекомендации, на основе финальной метрики
recommend_for_user = most_interesting.sort_values(by='common_rating', ascending=False)[:5][['title', 'common_rating']]
recommend_for_user

,title,common_rating
656,"Godfather, The (1972)",4.414372
2211,Fight Club (1999),4.408199
45,"Usual Suspects, The (1995)",4.398905
275,"Shawshank Redemption, The (1994)",4.393601
916,"Godfather: Part II, The (1974)",4.359118
